In [1]:
# Define model parameters
learning_rate = 0.005
training_epochs = 5
display_step = 1
training_dropout = 0.90
n_samples = y_train.shape[0]
batch_size = 2048

# Define how many inputs and outputs are in our neural network
number_of_inputs = 30
number_of_outputs = 2

# Maintain fixed ratio of nodes between each layer
multiplier = 1.5

# Number of nodes in each layer
layer_1_nodes = 18
layer_2_nodes = round(layer_1_nodes*multiplier)
layer_3_nodes = round(layer_2_nodes*multiplier)

# Percent of nodes to keep during dropout
dropout_keep = tf.placeholder(tf.float32)

# Define input layer
with tf.variable_scope('input', reuse=tf.AUTO_REUSE):
    X = tf.placeholder(tf.float32, shape=(None, number_of_inputs))
    
# Layer 1
with tf.variable_scope('layer_1', reuse=tf.AUTO_REUSE):
    weights = tf.get_variable('weights1', shape=[number_of_inputs, layer_1_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable('biases1', shape=[layer_1_nodes], initializer=tf.zeros_initializer)
    layer_1_output = tf.nn.sigmoid(tf.matmul(X, weights) + biases) # matrix multiplication and a standard rectified linear unit 

# Layer 2
with tf.variable_scope('layer_2', reuse=tf.AUTO_REUSE):
    weights = tf.get_variable('weights2', shape=[layer_1_nodes, layer_2_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable('biases2', shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.sigmoid(tf.matmul(layer_1_output, weights) + biases)
    
# Layer 3
with tf.variable_scope('layer_3', reuse=tf.AUTO_REUSE):
    weights = tf.get_variable('weights3', shape=[layer_2_nodes, layer_3_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable('biases3', shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)
    layer_3_output = tf.nn.dropout(layer_3_output, rate=dropout_keep) # keep_prob will be deprecated
    
# Layer 4 
with tf.variable_scope('layer_4', reuse=tf.AUTO_REUSE):
    weights = tf.get_variable('weights4', shape=[layer_3_nodes, number_of_outputs], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable('biases4', shape=[number_of_outputs], initializer=tf.zeros_initializer())
    layer_4_output = tf.nn.softmax(tf.matmul(layer_3_output, weights) + biases)

# # Output
# y = layer_4_output
# y_ = tf.placeholder(tf.float32, shape=(None, number_of_outputs))

# # Cost function
# cost = -tf.reduce_sum(y_*tf.log(y))

# # Optimizer
# optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# Define the cost function
with tf.variable_scope('cost', reuse=tf.AUTO_REUSE):
    Y = tf.placeholder(tf.float32, shape=(batch_size, None))
    cost = -tf.reduce_sum(layer_4_output*tf.log(Y)) # ???
    
# Define the optimizer function (train & optimize)
with tf.variable_scope('train', reuse=tf.AUTO_REUSE):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
# Correct prediction from softmax
correct_prediction = tf.equal(tf.argmax(layer_4_output, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

accuracy_summary = [] 
cost_summary = [] 
valid_accuracy_summary = [] 
valid_cost_summary = [] 
stop_early = 0 

# Save weights location
checkpoint = '../Checkpoints/'
saver = tf.train.Saver(max_to_keep=1)

################################################
####### Initialize Session
################################################

with tf.Graph().as_default(), tf.Session() as session:
    
    # Initialize a training session after defining the model
    with tf.Session() as session:

        # Run the global variable initializer to initialize all variables/layers in the neural network
        session.run(tf.global_variables_initializer()) # executes commands by calling session.run()

        # Training
        for epoch in range(training_epochs):
            for batch in range(int(n_samples/batch_size)):
                batch_x = X_input[batch*batch_size: (1+batch)*batch_size]
                batch_y = Y_input[batch*batch_size: (1+batch)*batch_size]

                # Reshape
                batch_x = np.reshape(batch_x, (2048, -1))
                batch_y = np.reshape(batch_y, (2048, -1))

                # Run optimizer
                session.run('optimizer', feed_dict={X: batch_x, Y: batch_y})

            # Display logs after every epoch
            if (epoch) % display_step == 0:
                train_accuracy, newCost = sess.run([accuracy, cost], feed_dict={X: X_input, Y: Y_input})
                valid_accuracy, newCost = sess.run([accuracy, cost], feed_dict={X: X_input_valid, Y: Y_input_valid})

                print('Epoch: ', epoch,
                      'Acc = {:.5f}'.format(train_accuracy), 
                      'Cost = {:.5f}'.format(newCost),
                      'Valid_Acc = {:.5f}'.format(valid_accuracy), 
                      'Valid_Cost = {:.5f}'.format(valid_newCost))

                # Saving weights
                if epoch > 0 and valid_accuracy > max(valid_accuracy_summary) and valid_accuracy > 0.999:
                    saver.save(session, checkpoint)

                # Record the results of the model
                accuracy_summary.append(train_accuracy)
                cost_summary.append(newCost)
                valid_accuracy_summary.append(valid_accuracy)
                valid_cost_summary.append(valid_newCost)

                # If the model does not improve after 15 logs, stop the training
                if valid_accuracy < max(valid_accuracy_summary) and epoch > 100:
                    stop_early += 1
                    if stop_early == 15:
                        break
                else:
                    stop_early = 0

        print("Finished.")

In [ ]:
model1 = Sequential()

model1.add(Conv2D(32, 3, padding='same', input_shape=(256,256,3), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

#print("First layer...")
model1.add(Conv2D(32, 3, padding='same', activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

#print("Second layer...")
model1.add(Conv2D(64, 3, padding='same', activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

model1.add(Flatten()) 
model1.add(Dense(64))
model1.add(Activation('relu'))
model1.add(Dropout(0.5))
model1.add(Dense(1))
model1.add(Activation('sigmoid'))

model1.load_weights("Weights_model1.h5")

model2 = Sequential()

model2.add(Conv2D(32, 3, padding='same', input_shape=(256,256,3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

#print("First layer...")
model2.add(Conv2D(32, 3, padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

#print("Second layer...")
model2.add(Conv2D(64, 3, padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

model2.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model2.add(Dense(64))
model2.add(Activation('relu'))
model2.add(Dropout(0.5))
model2.add(Dense(1))
model2.add(Activation('sigmoid'))

model2.load_weights("Weights_model2.h5")

Training_water_data = 'C:\\Users\\ysmehta\\'+folder+'\\'
training_water_data = [x for x in sorted(os.listdir(Training_water_data))]
x_train_water_data = np.empty((len(training_water_data),256,256,3),dtype = 'float32')
for i,name  in enumerate(training_water_data):
    im = cv2.imread(Training_water_data + name).astype('int16').astype('float32')/255.
    im = im[:,:,:]
    im = cv2.resize(im,dsize = (256,256),interpolation = cv2.INTER_NEAREST)
    x_train_water_data[i] = im

x_train_water_data = x_train_water_data.reshape(x_train_water_data.shape[0],256,256,3)

for i in range(0,x_train_water_data.shape[0]):
    A = x_train_water_data[i]
    B = A.reshape(1,256,256,3)
    prediction1 = model1.predict(B,verbose=0)
    if (prediction1 >= 0.7):
    prediction2 = model2.predict(B, verbose=0)
else:
    returnstring="No manhole found"
    prediction2 = -1
if(prediction2 >= 0.6):
    returnstring="Water Leakage"

else:
    if(prediction1 >= 0.7):
        returnstring="Not a Leakage"
return returnstring;

if __name__ == "__main__":
    app.run(host='131.163.144.196',port=5000)`